In [1]:
import pandas as pd
import os
import numpy as np

* Cargar archivos

In [2]:
# CARGAR ARCHIVOS PARA MANEJO
def leer_archivos(ruta): 
    archivos = [f for f in os.listdir(ruta) if f.endswith(".csv")]

    dataframes = {}
    errores = {}

    for archivo in archivos:
        path = os.path.join(ruta, archivo)
        try:
            df = pd.read_csv(path, encoding='latin_1', low_memory=False)
            df.columns = df.columns.str.lower()
            dataframes[archivo] = df

        except Exception as e:
            errores[archivo] = str(e)

    # Mostrar archivos con errores
    if errores:
        print("\n⚠️ Archivos con errores al cargar:")
        for archivo, error in errores.items():
            print(f"   ❌ {archivo}: {error}")
    else:
        print("\n✅ Todos los archivos se cargaron correctamente.")
    i = 1
    for archivo in archivos:
        if archivo in dataframes:
            print(f"{i}  📄 {archivo}: {dataframes[archivo].shape[0]} filas, {dataframes[archivo].shape[1]} columnas") 
            i += 1
    
    return dataframes, archivos

In [3]:
def comparar_columnas(dataframes):

    columnas_info = {}
    resultados = {}

    # Extraer columnas y tipos
    for nombre, df in dataframes.items():
        columnas_info[nombre] = {
            "columnas": list(df.columns),
            "tipos": df.dtypes.to_dict()
        }

    # Usar el primer archivo como referencia
    referencia = list(dataframes.keys())[0]
    cols_ref = set(columnas_info[referencia]["columnas"])

    for nombre, info in columnas_info.items():
        if nombre == referencia:
            continue

        cols_actual = set(info["columnas"])
        comunes = cols_ref.intersection(cols_actual)
        distintas = cols_ref.symmetric_difference(cols_actual)

        print(f"\n📂 Comparando {referencia} vs {nombre}:")
        print(f"    ✔️ Columnas comunes ({len(comunes)}): {sorted(comunes)}")
        print(f"    ❌ Columnas distintas ({len(distintas)}): {sorted(distintas)}")

        if len(distintas) == 0:
            print("     ✅ Las columnas son idénticas")
        else:
            print(f"    🔴 Existen {len(distintas)} variables diferentes")


In [4]:
# Carpeta UNCLEAN
ruta = r"C:\Users\Hp\DENGUE\Data\raw_data\Colombia"
dataframes, archivos  = leer_archivos(ruta)


✅ Todos los archivos se cargaron correctamente.
1  📄 Datos_2013_210.csv: 122441 filas, 73 columnas
2  📄 Datos_2013_220.csv: 3113 filas, 63 columnas
3  📄 Datos_2014_210.csv: 105356 filas, 73 columnas
4  📄 Datos_2014_220.csv: 2619 filas, 63 columnas
5  📄 Datos_2015_210.csv: 95023 filas, 73 columnas
6  📄 Datos_2015_220.csv: 1421 filas, 63 columnas
7  📄 Datos_2016_210.csv: 100117 filas, 74 columnas
8  📄 Datos_2016_220.csv: 899 filas, 63 columnas
9  📄 Datos_2017_210.csv: 25048 filas, 74 columnas
10  📄 Datos_2017_220.csv: 236 filas, 63 columnas
11  📄 Datos_2018_210.csv: 43652 filas, 74 columnas
12  📄 Datos_2018_220.csv: 519 filas, 64 columnas
13  📄 Datos_2019_210.csv: 123641 filas, 73 columnas
14  📄 Datos_2019_220.csv: 1348 filas, 70 columnas
15  📄 Datos_2020_210.csv: 76419 filas, 73 columnas
16  📄 Datos_2020_220.csv: 862 filas, 74 columnas
17  📄 Datos_2021_210.csv: 49325 filas, 74 columnas
18  📄 Datos_2021_220.csv: 940 filas, 77 columnas
19  📄 Datos_2022_210.csv: 65691 filas, 73 columnas
2

In [5]:
comparar_columnas(dataframes)


📂 Comparando Datos_2013_210.csv vs Datos_2013_220.csv:
    ✔️ Columnas comunes (59): ['ajuste', 'ano', 'area', 'cbmte', 'cer_def', 'cod_ase', 'cod_dpto_n', 'cod_dpto_o', 'cod_dpto_r', 'cod_eve', 'cod_mun_n', 'cod_mun_o', 'cod_mun_r', 'cod_pais_o', 'cod_pre', 'cod_sub', 'con_fin', 'confirmados', 'consecutive', 'departamento_notificacion', 'departamento_ocurrencia', 'departamento_residencia', 'edad', 'fec_aju', 'fec_arc_xl', 'fec_con', 'fec_def', 'fec_hos', 'fec_not', 'fecha_nto', 'fm_fuerza', 'fm_grado', 'fm_unidad', 'gp_carcela', 'gp_desmovi', 'gp_desplaz', 'gp_discapa', 'gp_gestan', 'gp_indigen', 'gp_mad_com', 'gp_migrant', 'gp_otros', 'gp_pobicfb', 'gp_psiquia', 'gp_vic_vio', 'gru_pob', 'ini_sin', 'municipio_notificacion', 'municipio_ocurrencia', 'municipio_residencia', 'ocupacion', 'pac_hos', 'per_etn', 'semana', 'sexo', 'tip_cas', 'tip_ss', 'uni_med', 'va_sispro']
    ❌ Columnas distintas (18): ['cen_pobla', 'cod_pais_r', 'consecutive_origen', 'estado_final_de_caso', 'estrato', 'e

In [6]:
changes = {
   'con_fin': 'muerte',
   'departamento_ocurrencia': 'departamento',
   'fec_con': 'fecha_consulta',
   'fec_hos': 'fecha_hospitalizacion',
   'fec_not': 'fecha_notificacion',
   'fecha_nto': 'fecha_nacimiento',
   'ini_sin': 'fecha_inicio_sintomas',
   'municipio_ocurrencia': 'municipio',
   'pac_hos': 'hospitalizado',
   'per_etn': 'pertenencia_etnica',
   'tip_cas': 'tipo_caso',
   'tip_ss': 'regimen_salud',
}

In [7]:
# Renombrar 
log_renombrado = {}

for archivo in archivos:
    path = os.path.join(ruta, archivo)
    try:
        df = pd.read_csv(path, encoding='latin_1', low_memory=False)
        dataframes[archivo] = df
        df.columns = df.columns.str.lower()
        # Detectar columnas que se pueden renombrar
        columnas_presentes = df.columns.intersection(changes.keys())
        cambios_aplicados = {col: changes[col] for col in columnas_presentes}

        # Renombrar columnas
        df = df.rename(columns=cambios_aplicados)

        # Guardar el archivo sobrescrito
        df.to_csv(path, index=False, encoding='latin1')

        # Registrar cambios
        log_renombrado[archivo] = cambios_aplicados

        print(f"✔️ {archivo}: columnas renombradas → {list(cambios_aplicados.keys())}")

    except Exception as e:
        print(f"❌ Error en {archivo}: {e}")
        log_renombrado[archivo] = f"Error: {str(e)}"

print("\n✅ Renombrado completado. Archivos editados directamente.")


✔️ Datos_2013_210.csv: columnas renombradas → ['fec_not', 'tip_ss', 'per_etn', 'fec_con', 'ini_sin', 'tip_cas', 'pac_hos', 'fec_hos', 'con_fin', 'fecha_nto', 'departamento_ocurrencia', 'municipio_ocurrencia']
✔️ Datos_2013_220.csv: columnas renombradas → ['fec_not', 'tip_ss', 'per_etn', 'fec_con', 'ini_sin', 'tip_cas', 'pac_hos', 'fec_hos', 'con_fin', 'fecha_nto', 'departamento_ocurrencia', 'municipio_ocurrencia']
✔️ Datos_2014_210.csv: columnas renombradas → ['fec_not', 'tip_ss', 'per_etn', 'fec_con', 'ini_sin', 'tip_cas', 'pac_hos', 'fec_hos', 'con_fin', 'fecha_nto', 'departamento_ocurrencia', 'municipio_ocurrencia']
✔️ Datos_2014_220.csv: columnas renombradas → ['fec_not', 'tip_ss', 'per_etn', 'fec_con', 'ini_sin', 'tip_cas', 'pac_hos', 'fec_hos', 'con_fin', 'fecha_nto', 'departamento_ocurrencia', 'municipio_ocurrencia']
✔️ Datos_2015_210.csv: columnas renombradas → ['fec_not', 'tip_ss', 'per_etn', 'fec_con', 'ini_sin', 'tip_cas', 'pac_hos', 'fec_hos', 'con_fin', 'fecha_nto', 'depar

In [8]:
dataframes, archivos  = leer_archivos(ruta)
comparar_columnas(dataframes)


✅ Todos los archivos se cargaron correctamente.
1  📄 Datos_2013_210.csv: 122441 filas, 73 columnas
2  📄 Datos_2013_220.csv: 3113 filas, 63 columnas
3  📄 Datos_2014_210.csv: 105356 filas, 73 columnas
4  📄 Datos_2014_220.csv: 2619 filas, 63 columnas
5  📄 Datos_2015_210.csv: 95023 filas, 73 columnas
6  📄 Datos_2015_220.csv: 1421 filas, 63 columnas
7  📄 Datos_2016_210.csv: 100117 filas, 74 columnas
8  📄 Datos_2016_220.csv: 899 filas, 63 columnas
9  📄 Datos_2017_210.csv: 25048 filas, 74 columnas
10  📄 Datos_2017_220.csv: 236 filas, 63 columnas
11  📄 Datos_2018_210.csv: 43652 filas, 74 columnas
12  📄 Datos_2018_220.csv: 519 filas, 64 columnas
13  📄 Datos_2019_210.csv: 123641 filas, 73 columnas
14  📄 Datos_2019_220.csv: 1348 filas, 70 columnas
15  📄 Datos_2020_210.csv: 76419 filas, 73 columnas
16  📄 Datos_2020_220.csv: 862 filas, 74 columnas
17  📄 Datos_2021_210.csv: 49325 filas, 74 columnas
18  📄 Datos_2021_220.csv: 940 filas, 77 columnas
19  📄 Datos_2022_210.csv: 65691 filas, 73 columnas
2

* ELIMINAR

In [9]:
delete = [
    'ajuste', 'cbmte', 'cer_def', 'cod_ase', 'cod_dpto_n', 'cod_dpto_r', 
    'cod_mun_n', 'cod_mun_r', 'cod_pais_o', 'cod_pais_r', 
    'cod_pre', 'cod_sub', 'confirmados', 'consecutive', 'consecutive_origen', 
    'departamento_notificacion', 'departamento_residencia', 
    'fec_aju', 'fec_arc_xl', 'fec_def', 
    'fm_fuerza', 'fm_grado', 'fm_unidad', 'fuente', 
    'gp_carcela', 'gp_desmovi', 'gp_desplaz', 'gp_discapa', 'gp_gestan', 
    'gp_indigen', 'gp_mad_com', 'gp_migrant', 'gp_otros', 'gp_pobicfb', 
    'gp_psiquia', 'gp_vic_vio', 'gru_pob', 
    'municipio_notificacion', 'municipio_residencia', 'nacionalidad', 
    'nom_grupo', 'nom_upgd', 'nombre_nacionalidad', 'ocupacion', 
    'pais_ocurrencia', 'pais_residencia', 
    'sem_ges','va_sispro', 
    'consecutive', 'consecutive_12', 
    'cen_pobla', 'localidad', 'nom_est_f_caso', 'nombre_upgd', 
    'cod_eve.1',
    'partición', 
    'bar_ver', 
    'vereda', 'version',
    'estado_final_de_caso', 'estrato', 'evento', 'nombre_evento'
]

In [11]:
# Eliminar archivos
log_eliminacion = {}

for archivo in archivos:
    path_csv = os.path.join(ruta, archivo)
    try:
        df = pd.read_csv(path_csv, encoding='latin1', low_memory=False)
        df.columns = df.columns.str.lower()
        # Detectar columnas presentes que se van a eliminar
        cols_a_eliminar = [c for c in delete if c in df.columns]

        # Eliminar columnas
        df = df.drop(columns=cols_a_eliminar, errors="ignore")

        # Guardar sobrescribiendo el archivo
        df.to_csv(path_csv, index=False, encoding='latin1')

        # Registrar cambios
        log_eliminacion[archivo] = cols_a_eliminar
        print(f"✔️ {archivo}: columnas eliminadas → {cols_a_eliminar}")

    except Exception as e:
        log_eliminacion[archivo] = f"Error: {str(e)}"
        print(f"❌ Error en {archivo}: {e}")

print("\n✅ Eliminación completada. Archivos editados directamente.")

✔️ Datos_2013_210.csv: columnas eliminadas → ['ajuste', 'cbmte', 'cer_def', 'cod_ase', 'cod_dpto_n', 'cod_dpto_r', 'cod_mun_n', 'cod_mun_r', 'cod_pais_o', 'cod_pais_r', 'cod_pre', 'cod_sub', 'confirmados', 'consecutive', 'consecutive_origen', 'departamento_notificacion', 'departamento_residencia', 'fec_aju', 'fec_arc_xl', 'fec_def', 'fm_fuerza', 'fm_grado', 'fm_unidad', 'fuente', 'gp_carcela', 'gp_desmovi', 'gp_desplaz', 'gp_discapa', 'gp_gestan', 'gp_indigen', 'gp_mad_com', 'gp_migrant', 'gp_otros', 'gp_pobicfb', 'gp_psiquia', 'gp_vic_vio', 'gru_pob', 'municipio_notificacion', 'municipio_residencia', 'nacionalidad', 'nom_grupo', 'nom_upgd', 'nombre_nacionalidad', 'ocupacion', 'pais_ocurrencia', 'pais_residencia', 'sem_ges', 'va_sispro', 'consecutive', 'nom_est_f_caso', 'estado_final_de_caso', 'estrato', 'nombre_evento']
✔️ Datos_2013_220.csv: columnas eliminadas → ['ajuste', 'cbmte', 'cer_def', 'cod_ase', 'cod_dpto_n', 'cod_dpto_r', 'cod_mun_n', 'cod_mun_r', 'cod_pais_o', 'cod_pre', '

In [19]:
dataframes, archivos  = leer_archivos(ruta)
comparar_columnas(dataframes)


✅ Todos los archivos se cargaron correctamente.
1  📄 Datos_2013_210.csv: 122441 filas, 21 columnas
2  📄 Datos_2013_220.csv: 3113 filas, 21 columnas
3  📄 Datos_2014_210.csv: 105356 filas, 21 columnas
4  📄 Datos_2014_220.csv: 2619 filas, 21 columnas
5  📄 Datos_2015_210.csv: 95023 filas, 21 columnas
6  📄 Datos_2015_220.csv: 1421 filas, 21 columnas
7  📄 Datos_2016_210.csv: 100117 filas, 21 columnas
8  📄 Datos_2016_220.csv: 899 filas, 21 columnas
9  📄 Datos_2017_210.csv: 25048 filas, 21 columnas
10  📄 Datos_2017_220.csv: 236 filas, 21 columnas
11  📄 Datos_2018_210.csv: 43652 filas, 21 columnas
12  📄 Datos_2018_220.csv: 519 filas, 21 columnas
13  📄 Datos_2019_210.csv: 123641 filas, 21 columnas
14  📄 Datos_2019_220.csv: 1348 filas, 21 columnas
15  📄 Datos_2020_210.csv: 76419 filas, 21 columnas
16  📄 Datos_2020_220.csv: 862 filas, 21 columnas
17  📄 Datos_2021_210.csv: 49325 filas, 21 columnas
18  📄 Datos_2021_220.csv: 940 filas, 21 columnas
19  📄 Datos_2022_210.csv: 65691 filas, 21 columnas
2

* UNIR

In [20]:
dataframes, archivos = leer_archivos(ruta)

# Unir todos en un solo DataFrame
df_final = pd.concat(dataframes.values(), ignore_index=True)
df_final.info()


✅ Todos los archivos se cargaron correctamente.
1  📄 Datos_2013_210.csv: 122441 filas, 21 columnas
2  📄 Datos_2013_220.csv: 3113 filas, 21 columnas
3  📄 Datos_2014_210.csv: 105356 filas, 21 columnas
4  📄 Datos_2014_220.csv: 2619 filas, 21 columnas
5  📄 Datos_2015_210.csv: 95023 filas, 21 columnas
6  📄 Datos_2015_220.csv: 1421 filas, 21 columnas
7  📄 Datos_2016_210.csv: 100117 filas, 21 columnas
8  📄 Datos_2016_220.csv: 899 filas, 21 columnas
9  📄 Datos_2017_210.csv: 25048 filas, 21 columnas
10  📄 Datos_2017_220.csv: 236 filas, 21 columnas
11  📄 Datos_2018_210.csv: 43652 filas, 21 columnas
12  📄 Datos_2018_220.csv: 519 filas, 21 columnas
13  📄 Datos_2019_210.csv: 123641 filas, 21 columnas
14  📄 Datos_2019_220.csv: 1348 filas, 21 columnas
15  📄 Datos_2020_210.csv: 76419 filas, 21 columnas
16  📄 Datos_2020_220.csv: 862 filas, 21 columnas
17  📄 Datos_2021_210.csv: 49325 filas, 21 columnas
18  📄 Datos_2021_220.csv: 940 filas, 21 columnas
19  📄 Datos_2022_210.csv: 65691 filas, 21 columnas
2

In [21]:
df_final = df_final[sorted(df_final.columns)]
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948142 entries, 0 to 948141
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   ano                    948142 non-null  int64 
 1   area                   948142 non-null  int64 
 2   cod_dpto_o             948142 non-null  int64 
 3   cod_eve                948142 non-null  int64 
 4   cod_mun_o              948142 non-null  int64 
 5   departamento           948142 non-null  object
 6   edad                   948142 non-null  int64 
 7   fecha_consulta         948049 non-null  object
 8   fecha_hospitalizacion  400963 non-null  object
 9   fecha_inicio_sintomas  948089 non-null  object
 10  fecha_nacimiento       918020 non-null  object
 11  fecha_notificacion     948142 non-null  object
 12  hospitalizado          948142 non-null  int64 
 13  muerte                 948142 non-null  int64 
 14  municipio              948142 non-null  object
 15  

In [22]:
count_duplicados = df_final.duplicated().sum()
print(f"Número de filas duplicadas exactas en columnas de interés: {count_duplicados}")

Número de filas duplicadas exactas en columnas de interés: 332


In [23]:
df_limpio = df_final.drop_duplicates(keep="first").copy()
df_final = df_limpio
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 947810 entries, 0 to 948141
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   ano                    947810 non-null  int64 
 1   area                   947810 non-null  int64 
 2   cod_dpto_o             947810 non-null  int64 
 3   cod_eve                947810 non-null  int64 
 4   cod_mun_o              947810 non-null  int64 
 5   departamento           947810 non-null  object
 6   edad                   947810 non-null  int64 
 7   fecha_consulta         947717 non-null  object
 8   fecha_hospitalizacion  400814 non-null  object
 9   fecha_inicio_sintomas  947757 non-null  object
 10  fecha_nacimiento       917713 non-null  object
 11  fecha_notificacion     947810 non-null  object
 12  hospitalizado          947810 non-null  int64 
 13  muerte                 947810 non-null  int64 
 14  municipio              947810 non-null  object
 15  perte

In [24]:
# Crear evento a partir del codigo 
# evento = Clasifico if code_eve = 210 
# evento = Grave if code_eve = 220
df_final['evento'] = np.where(df_final['cod_eve'] == 210, 'Clasico', 
                       np.where(df_final['cod_eve'] == 220, 'Grave', 'Otros'))
df_final['evento'].unique()

array(['Clasico', 'Grave'], dtype=object)

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 947810 entries, 0 to 948141
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   ano                    947810 non-null  int64 
 1   area                   947810 non-null  int64 
 2   cod_dpto_o             947810 non-null  int64 
 3   cod_eve                947810 non-null  int64 
 4   cod_mun_o              947810 non-null  int64 
 5   departamento           947810 non-null  object
 6   edad                   947810 non-null  int64 
 7   fecha_consulta         947717 non-null  object
 8   fecha_hospitalizacion  400814 non-null  object
 9   fecha_inicio_sintomas  947757 non-null  object
 10  fecha_nacimiento       917713 non-null  object
 11  fecha_notificacion     947810 non-null  object
 12  hospitalizado          947810 non-null  int64 
 13  muerte                 947810 non-null  int64 
 14  municipio              947810 non-null  object
 15  perte

In [27]:
import os

ruta_salida = r"C:\Users\Hp\DENGUE\Data"
archivo = "data_colombia.csv"

# Crear ruta completa
path_out = os.path.join(ruta_salida, archivo)

# Guardar
df_final.to_csv(path_out, index=False, encoding="latin1")
print(f"✔️ Guardado en: {path_out}")


✔️ Guardado en: C:\Users\Hp\DENGUE\Data\data_colombia.csv
